In [1]:
import numpy as np
import pandas as pd
import requests
import json
from typing import Optional, Union, List

### step1
1. 用户提问，传递为q1
2. 返回匹配结果：是一个数据框。rep_id代表的是知识条目的index。mrr是计算指标（越大的mrr，代表和用户的提问越相似）

In [2]:

def search_query(q1: str, topn: int = 2) -> pd.DataFrame:
    # q1 = "今天好热"
    web = requests.get(url="http://0.0.0.0:8010/search_topn",
                       params={'search_text': q1, 'topn': topn})
    res = pd.DataFrame(json.loads(web.json()))
    return res


search_query(q1='你天气热不热')

,rep_id,mrr
0,17dad48d-edb9-425a-b7f5-c65afd695a52,0.611111


### step2
1. 把上面的表的第一行的rep_id拿到，使用rep_id获得查询到的内容（返回标题列表，和相似问法列表）

In [3]:
rep_id = search_query(q1='附近吃的')['rep_id'].tolist()[0]


def get_title_simi(rep_id) -> Union[pd.DataFrame, pd.DataFrame]:
    web = requests.get(url="http://0.0.0.0:8010/select_by_repid",
                       params={'rep_id': rep_id})
    # json.loads(web.json())
    title_df = pd.DataFrame(json.loads(web.json().get('title')))
    simi_df = pd.DataFrame(json.loads(web.json().get('sim')))
    return title_df, simi_df


td, sd = get_title_simi(rep_id)
# print(td)
# print(sd)

In [4]:
td
# web.json()

,title,answer,create_time,modify_time,rep_id,status_question
0,你是谁,由公众号【world of statistics】作者创建,2022-08-19 19:59:01.103599,2022-08-19 19:59:01.104775,17dad48d-edb9-425a-b7f5-c65afd695a52,1


In [5]:
sd

,similarity,create_time,modify_time,rep_id,status_similar,sim_index
0,你叫什么名字,2022-08-19 19:59:01.105241,2022-08-19 19:59:01.105361,17dad48d-edb9-425a-b7f5-c65afd695a52,1,0
1,你从哪里来的,2022-08-19 19:59:01.105241,2022-08-19 19:59:01.105361,17dad48d-edb9-425a-b7f5-c65afd695a52,1,1


### step3
1. 有时候，需要把上面两个步骤连接在一起，同时，直接给到查询的答案。
2. 下面直接给到一个完整的流程


In [2]:
import numpy as np
import pandas as pd
import requests
import json
from typing import Optional, Union, List


# step1
def search_query(q1: str, topn: int = 2) -> pd.DataFrame:
    # q1 = "今天好热"
    web = requests.get(url="http://0.0.0.0:8010/search_topn",
                       params={'search_text': q1, 'topn': topn})
    res = pd.DataFrame(json.loads(web.json()))
    return res


# step2
def get_title_simi(rep_id) -> Union[pd.DataFrame, pd.DataFrame]:
    web = requests.get(url="http://0.0.0.0:8010/select_by_repid",
                       params={'rep_id': rep_id})
    # json.loads(web.json())
    title_df = pd.DataFrame(json.loads(web.json().get('title')))
    simi_df = pd.DataFrame(json.loads(web.json().get('sim')))
    return title_df, simi_df


# step3
def get_data(q1):
    rep_id = search_query(q1=q1)['rep_id'].tolist()[0]
    td, sd = get_title_simi(rep_id)
    res = td['answer'].tolist()[0]
    return res

# example
get_data('who you are')

JSONDecodeError: [Errno Expecting value] Internal Server Error: 0